# CTA Folder Analysis: Filter and Extract Metadata

This notebook scans CTA folders to find those with >100 DICOM images and extracts their metadata for analysis.

## 1. Import Required Libraries

In [51]:
import os
import json
from pathlib import Path
import pandas as pd
from collections import defaultdict
import warnings

# Try to import pydicom for DICOM metadata extraction
try:
    import pydicom
    HAS_PYDICOM = True
    print("✓ pydicom available for DICOM metadata extraction")
except ImportError:
    HAS_PYDICOM = False
    print("⚠ pydicom not available. Install with: pip install pydicom")
    print("  Metadata extraction will be limited to basic file info")

warnings.filterwarnings('ignore')

✓ pydicom available for DICOM metadata extraction


## 2. Set Base Directory Path

In [52]:
# Configuration
BASE_CTA_DIR = Path("mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA")
MIN_IMAGES_THRESHOLD = 100

# Verify base directory exists
if BASE_CTA_DIR.exists():
    print(f"✓ Base CTA directory found: {BASE_CTA_DIR}")
    print(f"✓ Filtering folders with >{MIN_IMAGES_THRESHOLD} images")
    print(f"✓ Will analyze CTA DICOM series folders")
else:
    print(f"✗ Base CTA directory not found: {BASE_CTA_DIR}")
    print("Please check the path and try again.")

# Common DICOM file extensions
DICOM_EXTENSIONS = {'.dcm', '.dicom', '.dic', '.DCM', '.DICOM', '.DIC', ''}  # Include files without extension

✓ Base CTA directory found: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA
✓ Filtering folders with >100 images
✓ Will analyze CTA DICOM series folders


## 3. Scan CTA Folders for Image Count

In [54]:
def is_dicom_file(file_path):
    """Check if file is likely a DICOM file based on extension or content"""
    if file_path.suffix.lower() in DICOM_EXTENSIONS:
        return True
    
    # For files without extension, try to read DICOM header
    if HAS_PYDICOM and file_path.suffix == '':
        try:
            pydicom.dcmread(str(file_path), stop_before_pixels=True)
            return True
        except:
            return False
    return False

def count_dicom_files(folder_path):
    """Count DICOM files in a folder"""
    count = 0
    try:
        for file_path in folder_path.iterdir():
            if file_path.is_file() and is_dicom_file(file_path):
                count += 1
    except Exception as e:
        print(f"Error scanning {folder_path}: {e}")
    return count

# Scan all CTA DICOM series folders (go two levels deep)
folder_stats = {}
print("Scanning CTA DICOM series folders...")

if BASE_CTA_DIR.exists():
    # The DICOM series are nested deeper, so we need to look in subdirectories
    for study_folder in BASE_CTA_DIR.iterdir():
        if study_folder.is_dir():
            print(f"Scanning study folder: {study_folder.name}")
            for series_folder in study_folder.iterdir():
                if series_folder.is_dir():
                    dicom_count = count_dicom_files(series_folder)
                    folder_stats[str(series_folder)] = dicom_count
                    print(f"  {series_folder.name}: {dicom_count} files")

print(f"\nFound {len(folder_stats)} DICOM series folders total")

Scanning CTA DICOM series folders...
Scanning study folder: 1.3.46.670589.61.128.0.20250519182905567
  1.3.46.670589.61.128.1.2025051918423275700050001: 12 files
  1.3.46.670589.61.128.1.2025051918423277100060001: 69 files
  1.3.46.670589.50.2.27943982313712160072.3129725653237519767: 25 files
  1.3.46.670589.61.128.1.2025051918351580100010001: 2 files
  1.3.46.670589.61.128.1.2025051918414792200040001: 1 files
  1.3.46.670589.61.128.1.2025051918423278000060002: 454 files
  1.3.46.670589.61.128.1.2025051918360893900020002: 507 files
  1.3.46.670589.61.128.1.2025051918423280300080001: 73 files
  1.3.46.670589.61.128.1.2025051918423281100080002: 481 files
  1.3.46.670589.61.128.1.2025051918423279500070002: 481 files
  1.3.46.670589.61.9527.1.20250519184313316.14923: 1 files
  1.3.46.670589.61.128.1.2025051918423278800070001: 73 files
  1.3.46.670589.61.128.1.2025051918360892800020001: 77 files
  1.3.46.670589.61.128.1.2025051918411727100030001: 1 files

Found 14 DICOM series folders tota

## 4. Filter Folders with More Than 100 Images

In [55]:
# Filter folders with >100 images
qualifying_folders = {folder: count for folder, count in folder_stats.items() 
                     if count > MIN_IMAGES_THRESHOLD}

print(f"🎯 Found {len(qualifying_folders)} folders with >{MIN_IMAGES_THRESHOLD} images:")
print("=" * 60)

if qualifying_folders:
    # Sort by image count (descending)
    sorted_folders = sorted(qualifying_folders.items(), key=lambda x: x[1], reverse=True)
    
    for folder_path, count in sorted_folders:
        folder_name = Path(folder_path).name
        print(f"📁 {folder_name}")
        print(f"   Path: {folder_path}")
        print(f"   Images: {count}")
        print()
else:
    print(f"No folders found with >{MIN_IMAGES_THRESHOLD} images")
    print("All folder counts:", folder_stats)

🎯 Found 4 folders with >100 images:
📁 1.3.46.670589.61.128.1.2025051918360893900020002
   Path: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA/1.3.46.670589.61.128.0.20250519182905567/1.3.46.670589.61.128.1.2025051918360893900020002
   Images: 507

📁 1.3.46.670589.61.128.1.2025051918423281100080002
   Path: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA/1.3.46.670589.61.128.0.20250519182905567/1.3.46.670589.61.128.1.2025051918423281100080002
   Images: 481

📁 1.3.46.670589.61.128.1.2025051918423279500070002
   Path: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA/1.3.46.670589.61.128.0.20250519182905567/1.3.46.670589.61.128.1.2025051918423279500070002
   Images: 481

📁 1.3.46.670589.61.128.1.2025051918423278000060002
   Path: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA/1.3.46.670589.61.128.0.20250519182905567/1.3.46.670589.61.128.1.2025051918423278000060002
   Images: 454



## 5. Extract DICOM Metadata

In [56]:
def extract_dicom_metadata(folder_path, sample_size=5):
    """Extract metadata from sample DICOM files in folder"""
    folder_path = Path(folder_path)
    metadata = {
        'folder_name': folder_path.name,
        'folder_path': str(folder_path),
        'total_files': folder_stats.get(str(folder_path), 0),
        'sample_files': [],
        'common_metadata': {}
    }
    
    if not HAS_PYDICOM:
        metadata['error'] = 'pydicom not available'
        return metadata
    
    # Get sample files
    dicom_files = [f for f in folder_path.iterdir() 
                   if f.is_file() and is_dicom_file(f)]
    
    if not dicom_files:
        metadata['error'] = 'No DICOM files found'
        return metadata
    
    sample_files = dicom_files[:sample_size]
    
    # Extract metadata from each sample file
    all_metadata = []
    for dicom_file in sample_files:
        try:
            ds = pydicom.dcmread(str(dicom_file), stop_before_pixels=True)
            
            file_meta = {
                'filename': dicom_file.name,
                'series_description': getattr(ds, 'SeriesDescription', 'N/A'),
                'study_description': getattr(ds, 'StudyDescription', 'N/A'),
                'modality': getattr(ds, 'Modality', 'N/A'),
                'patient_id': getattr(ds, 'PatientID', 'N/A'),
                'study_date': getattr(ds, 'StudyDate', 'N/A'),
                'series_number': getattr(ds, 'SeriesNumber', 'N/A'),
                'instance_number': getattr(ds, 'InstanceNumber', 'N/A'),
                'rows': getattr(ds, 'Rows', 'N/A'),
                'columns': getattr(ds, 'Columns', 'N/A'),
                'pixel_spacing': getattr(ds, 'PixelSpacing', 'N/A'),
                'slice_thickness': getattr(ds, 'SliceThickness', 'N/A'),
                'kvp': getattr(ds, 'KVP', 'N/A'),
                'exposure_time': getattr(ds, 'ExposureTime', 'N/A'),
                'manufacturer': getattr(ds, 'Manufacturer', 'N/A'),
                'manufacturer_model': getattr(ds, 'ManufacturerModelName', 'N/A')
            }
            
            all_metadata.append(file_meta)
            metadata['sample_files'].append(file_meta)
            
        except Exception as e:
            metadata['sample_files'].append({
                'filename': dicom_file.name,
                'error': str(e)
            })
    
    # Extract common metadata (fields that are same across samples)
    if all_metadata:
        first_meta = all_metadata[0]
        for key, value in first_meta.items():
            if key != 'filename' and key != 'instance_number':
                # Check if this value is consistent across all samples
                if all(meta.get(key) == value for meta in all_metadata):
                    metadata['common_metadata'][key] = value
    
    return metadata

# Extract metadata for all qualifying folders
print("🔍 Extracting DICOM metadata...")
folder_metadata = {}

for folder_path in qualifying_folders.keys():
    print(f"Processing: {Path(folder_path).name}")
    metadata = extract_dicom_metadata(folder_path)
    folder_metadata[folder_path] = metadata

🔍 Extracting DICOM metadata...
Processing: 1.3.46.670589.61.128.1.2025051918423278000060002
Processing: 1.3.46.670589.61.128.1.2025051918360893900020002
Processing: 1.3.46.670589.61.128.1.2025051918423281100080002
Processing: 1.3.46.670589.61.128.1.2025051918423279500070002


## 6. Generate Summary Report

In [57]:
# Create summary table
summary_data = []

for folder_path, metadata in folder_metadata.items():
    common = metadata.get('common_metadata', {})
    
    row = {
        'Folder Name': metadata['folder_name'],
        'Image Count': metadata['total_files'],
        'Series Description': common.get('series_description', 'N/A'),
        'Study Description': common.get('study_description', 'N/A'),
        'Modality': common.get('modality', 'N/A'),
        'Patient ID': common.get('patient_id', 'N/A'),
        'Study Date': common.get('study_date', 'N/A'),
        'Image Size': f"{common.get('rows', 'N/A')}x{common.get('columns', 'N/A')}",
        'Slice Thickness': common.get('slice_thickness', 'N/A'),
        'Manufacturer': common.get('manufacturer', 'N/A'),
        'Model': common.get('manufacturer_model', 'N/A')
    }
    summary_data.append(row)

# Display summary table
if summary_data:
    df_summary = pd.DataFrame(summary_data)
    print("📊 SUMMARY TABLE - CTA Folders with >100 Images")
    print("=" * 80)
    print(df_summary.to_string(index=False))
else:
    print("No qualifying folders found for summary table.")

📊 SUMMARY TABLE - CTA Folders with >100 Images
                                     Folder Name  Image Count Series Description Study Description Modality Patient ID Study Date Image Size  Slice Thickness Manufacturer       Model
1.3.46.670589.61.128.1.2025051918423278000060002          454         Angio THIN       BRAIN+ANGIO       CT 1000217055   20250519    512x512              1.0      Philips Incisive CT
1.3.46.670589.61.128.1.2025051918360893900020002          507         Plain Thin       BRAIN+ANGIO       CT 1000217055   20250519    512x512              1.0      Philips Incisive CT
1.3.46.670589.61.128.1.2025051918423281100080002          481       3rd Run Thin       BRAIN+ANGIO       CT 1000217055   20250519    512x512              1.0      Philips Incisive CT
1.3.46.670589.61.128.1.2025051918423279500070002          481       2nd Run Thin       BRAIN+ANGIO       CT 1000217055   20250519    512x512              1.0      Philips Incisive CT


In [58]:
# Detailed report for the folder with most images (likely the best one)
print("\n" + "="*80)
print("🏆 DETAILED METADATA FOR FOLDER WITH MOST IMAGES")
print("="*80)

if folder_metadata:
    # Find folder with most images
    best_folder = max(folder_metadata.items(), key=lambda x: x[1]['total_files'])
    best_path, best_meta = best_folder
    
    print(f"📁 Best Folder: {best_meta['folder_name']}")
    print(f"📊 Image Count: {best_meta['total_files']}")
    print(f"📍 Path: {best_meta['folder_path']}")
    
    print(f"\n🔍 Common Metadata (consistent across samples):")
    if best_meta.get('common_metadata'):
        for key, value in best_meta['common_metadata'].items():
            print(f"  • {key}: {value}")
    else:
        print("  No common metadata extracted")
    
    print(f"\n📋 Sample Files Analysis:")
    for i, sample in enumerate(best_meta.get('sample_files', [])[:3], 1):
        print(f"  Sample {i}: {sample.get('filename', 'Unknown')}")
        if 'error' in sample:
            print(f"    ❌ Error: {sample['error']}")
        else:
            print(f"    Series: {sample.get('series_description', 'N/A')}")
            print(f"    Instance: {sample.get('instance_number', 'N/A')}")
    
    # Export detailed JSON
    output_file = Path("cta_metadata_report.json")
    with open(output_file, 'w') as f:
        json.dump(folder_metadata, f, indent=2, default=str)
    print(f"\n💾 Detailed metadata exported to: {output_file}")
    
else:
    print("No metadata available to display.")


🏆 DETAILED METADATA FOR FOLDER WITH MOST IMAGES
📁 Best Folder: 1.3.46.670589.61.128.1.2025051918360893900020002
📊 Image Count: 507
📍 Path: mRs-prediction-3patient/sayyad habib CTA/sayyad habib CTA/1.3.46.670589.61.128.0.20250519182905567/1.3.46.670589.61.128.1.2025051918360893900020002

🔍 Common Metadata (consistent across samples):
  • series_description: Plain Thin
  • study_description: BRAIN+ANGIO
  • modality: CT
  • patient_id: 1000217055
  • study_date: 20250519
  • series_number: 3
  • rows: 512
  • columns: 512
  • pixel_spacing: [0.566406, 0.566406]
  • slice_thickness: 1
  • kvp: 120
  • exposure_time: 13764
  • manufacturer: Philips
  • manufacturer_model: Incisive CT

📋 Sample Files Analysis:
  Sample 1: Image_119.dcm
    Series: Plain Thin
    Instance: 120
  Sample 2: Image_131.dcm
    Series: Plain Thin
    Instance: 132
  Sample 3: Image_125.dcm
    Series: Plain Thin
    Instance: 126

💾 Detailed metadata exported to: cta_metadata_report.json


## 7. Recommendations

Based on the analysis above:

1. **Folders with >100 images** are most likely to contain complete imaging series suitable for analysis
2. **The folder with the highest image count** is typically the primary acquisition series
3. **Check Series Description** to identify the specific type of CTA scan (e.g., arterial phase, venous phase)
4. **Verify image dimensions and slice thickness** for consistency with your analysis requirements

### Next Steps:
- Use the folder with the most images for your primary analysis
- Verify DICOM metadata shows consistent acquisition parameters
- Consider the modality and series description to ensure it matches your research needs